What is the GRASP principles?
----

It is a set of 9 must know principles when writing code that is understandable, maintainable and extensible. GRASP Stands for General Responsibility Assingment Software Patterns.

Creator Principle
----
The Creator principle gives guidelines as to which class B should be in charge of creating a certain type of objects A.

For example:
 - B contains or aggregates instances of A
 - B closely uses A
 - B has the inputs to construct A

In the example below, we can see that Sale is the most suitable creator class for SaleLineItem since it is closely related to it and has the inputs to construct it.

```Python

# Before
from dataclasses import dataclass, field
from datetime import datetime


@dataclass
class ProductDescription:
    price: int
    description: str


@dataclass
class SaleLineItem:
    product: ProductDescription
    quantity: int


@dataclass
class Sale:
    items: list[SaleLineItem] = field(default_factory=list)
    time: datetime = field(default=datetime.now())


# After
@dataclass
class Sale:
    items: list[SaleLineItem] = field(default_factory=list)
    time: datetime = field(default=datetime.now())

    def add_item(self, product: ProductDescription, quantity: int):
        self.items.append(SaleLineItem(product, quantity))
```

Information Expert
----
This principle recommends that if you have an operation to do, and this operations needs inputs, then you should consider putting the responsibility of carrying out this operation in the class that contains the inputs for it. This principle enables you to decide where you should assign new responsibility. It should be as close as possible to the part that has the knowledge to fulfill the responsibility


For example, lets say that we want to create the total price for a particular sales. Where should it fall under? In our contrive example, the best place would be the SalesLineItem as it has all the information necessary to derive the total price.

```Python
# Before
@dataclass
class ProductDescription:
    price: int
    description: str


@dataclass
class SaleLineItem:
    product: ProductDescription
    quantity: int

# After
@dataclass
class SaleLineItem:
    product: ProductDescription
    quantity: int

    def total_price(self):
        return self.product.price * self.quantity

```

Controller
----
It separate from the user interface class with the data storage class. It acts as a binding mechanism between both class and performs the action. Some common examples could be a GUI Tkinter class with a UI Class as its view, and a data storage class as its model and a Controller class that binds the two together.


Protected Variations
----
The principle of Protected variations is related to the one of Low coupling, because it helps reducing the impacts of the changes of the code of one part A on another part B. The code of part B is protected against the variations of the code of part A, hence the name of the pattern.

```Python

# Before
class Oranges:
    def __init__(self, name, price):
        self.price = price

    def get_price(self):
        return self.price

class FruitBasket:
    def __init__(self, fruit: Oranges):
        self.fruit = fruit

    def get_fruit_price(self):
        return self.fruit.get_price()

# After
from typing import Protocol

class Fruit(Protocol):
    def get_price(self):
        ...

class Oranges(Fruit):
    def __init__(self, name, price):
        self.price = price

    def get_price(self):
        return self.price

class Apple(Fruit):
    def __init__(self, name, price):
        self.price = price

    def get_price(self):
        return self.price

class FruitBasket:
    def __init__(self, fruit: Fruit):
        self.fruit = fruit

    def get_fruit_price(self):
        return self.fruit.get_price()

```

Indirection
----
It introduce an intermediate unit across two other objects so that the direct coupling is not introduced. 
This principle helps to reduce coupling

Some of the common patterns which uses indirection are:
- Facade
- Adapter


In the example below, the discount class was created to act as the intermediary between the Order and the Customer class. This way, the Order class does not have to know the implementation details of the Customer class.

```Python

# Before
from __future__ import annotations

from dataclasses import dataclass
from datetime import date

YEAR = 360
HALF_YEAR = YEAR // 2

@dataclass
class Order:
    """Represents and order in an e-commerce system."""

    customer: Customer

    def get_discount(self) -> float:
        """Returns the discount for the order."""
        return self.customer.get_discount()


@dataclass
class Customer:
    """Represents a client."""

    since: date

    @property
    def lifetime_days(self) -> int:
        """Returns how long the person has been a customer in days."""
        return (date.today() - self.since).days

    def get_discount(self) -> float:
        """Returns the discount based on how long the person has been a client."""
        if self.lifetime_days < HALF_YEAR:
            return 0.0
        elif HALF_YEAR <= self.lifetime_days < YEAR:
            return 0.1
        elif YEAR <= self.lifetime_days < YEAR * 2:
            return 0.15
        else:
            return 0.2

# After
@dataclass
class Order:
    """Represents and order in an e-commerce system."""

    discount: Discount

    def get_discount(self):
        return self.discount.get_lifetime_discount()


@dataclass
class Discount:
    """Discount calculations."""

    customer: Customer

    def get_lifetime_discount(self) -> float:
        """Returns the discount based on how long the person has been a client."""
        if self.customer.lifetime_days < HALF_YEAR:
            return 0.0
        elif HALF_YEAR <= self.customer.lifetime_days < YEAR:
            return 0.1
        elif YEAR <= self.customer.lifetime_days < YEAR * 2:
            return 0.15
        else:
            return 0.2


@dataclass
class Customer:
    """Represents a client."""

    since: date

    @property
    def lifetime_days(self) -> int:
        """Returns how long the person has been a customer in days."""
        return (date.today() - self.since).days
```

Low Coupling
----

Coupling happens between two parts of the code when one depends on the other. Coupling introduces complexity, if only because the code can then no longer be understood isolation.

The design principle of low coupling encourages to keep coupling low, and it can help in particular to choose between two designs: select the one that introduces the lower amount of coupling.

Some of the common methods to reduce low coupling is by splitting the code into smaller parts, using interfaces, and using dependency injection.

High Cohesion
---
The principle of High cohesion encourages to focus classes around one responsibility, and to have all its components oriented towards achieving this responsibility. This is the principle of “do one thing and do it well”.

When you have low coupling, your code will tend to have high cohesion

Some tips to obtain high cohesion is as follows: 
- Dont have God Classes
- Dont have too many instance attributes
- Avoid having too many methods
- Either Data Focused or Behaviour Focused but the data structure pretty similar

Polymorphism
----
The usage for polymorphism is when there are several ways to accomplish a task, and you want to decouple the clients of this task from the various pieces of code that implement the various ways to perform it.

The Polymorphism principle is very close to the GoF Strategy pattern, if not identical. It contributes to the Low Coupling principle. It uses Inheritance to swap out behaviour



Pure Fabrication
----
Create a class that does not map to a domain object, and let it achieve this new responsibility in a cohesive way. 

In another word, when an object does not belong to an existint set of responsibilities, create a new module or separate class.

References
----
1) [Unleashing the power of Grasp Design Principles](https://zikazaki.medium.com/unleashing-the-power-of-grasp-design-principles-a-guide-to-building-robust-software-with-7a2e23e055a8#:~:text=In%20this%20article,%20we%20explored%20the%20fundamental%20GRASP)

2) [Grasp 9: Must-Know design principles](https://www.fluentcpp.com/2021/06/23/grasp-9-must-know-design-principles-for-code/)

3) [Apply the Grasp Design Principle to Improve your code](https://www.youtube.com/watch?v=fGNF6wuD-fg)